In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_cleaning import new_clean_data
from datetime import datetime
import plotly.graph_objs as go
import plotly.offline as pyo
import seaborn as sns
from  weather import getCleanWeather
%matplotlib inline

In [ ]:
weather = getCleanWeather(r'C:\Users\STH_0\Desktop\Data\Boston')

In [ ]:
bluebikes_idle_time = pd.read_csv(r"C:\Users\STH_0\Desktop\Data\bluebikes_idle_time_v4.csv")
bluebikes_idle_time['idle_time'] = pd.to_timedelta(bluebikes_idle_time['idle_time'])
bluebikes_idle_time['idle_time'] = bluebikes_idle_time['idle_time'].dt.total_seconds()
bluebikes_idle_time['idle_time'] = bluebikes_idle_time['idle_time'] / 60
bluebikes_idle_time['start_time'] = pd.to_datetime(bluebikes_idle_time['start_time'])
weather['timestamp'] = pd.to_datetime(weather['timestamp'])
bluebikes_idle_time.head()

In [ ]:
weather.head()

In [ ]:
merged_df = pd.merge_asof(bluebikes_idle_time.sort_values('start_time'), weather.sort_values('timestamp'),
                          left_on='start_time', right_on='timestamp',
                          direction='nearest')

In [ ]:
weather.info()

In [ ]:
# Convert 'start_time' column to datetime
merged_df['start_time'] = pd.to_datetime(merged_df['start_time'])
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'])

# Group the data by date and calculate the mean of idle time for each day
daily_idle_time_mean = merged_df.groupby(merged_df['start_time'].dt.date)['idle_time'].mean()
temperator_mean = merged_df.groupby(merged_df['timestamp'].dt.date)['temperature'].mean()

# Group the data by date and count the number of data points for each day
daily_data_count = merged_df.groupby(merged_df['start_time'].dt.date).size()

# Create the scatter plots
trace_idle_time = go.Scatter(x=daily_idle_time_mean.index, y=daily_idle_time_mean.values, name='Mean Idle Time')
trace_data_count = go.Scatter(x=temperator_mean.index, y=temperator_mean.values, name='Data Point Count')

# Set the layout
layout = go.Layout(title='Mean Idle Time and Data Point Count over 2 Years',
                   xaxis=dict(title='Date'), yaxis=dict(title='Value'))

# Create the figure
fig = go.Figure(data=[trace_idle_time, trace_data_count], layout=layout)

# Plot the figure
pyo.iplot(fig)

In [ ]:
sns.jointplot(x='temperature', y='idle_time', data=merged_df, kind='reg', color='g')

In [ ]:
#plotly temperature vs idle time
trace = go.Scatter(x=merged_df['temperature'], y=merged_df['idle_time'], mode='markers')
layout = go.Layout(title='Temperature vs Idle Time', xaxis=dict(title='Temperature'),
                     yaxis=dict(title='Idle Time'))
fig = go.Figure(data=[trace], layout=layout)
pyo.iplot(fig)
